In [ ]:
import pandas as pd
import numpy as np
import tkinter as tk
import warnings
import locale
import calendar
import chardet
import os
from tkinter import messagebox
from datetime import datetime
from unidecode import unidecode

warnings.filterwarnings('ignore')
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

rj_path = 'bases/gestiones/RJ'
clasa_path = 'bases/gestiones/CLASA'
mornese_path = 'bases/gestiones/MORNESE'

rj_archivos = [file for file in os.listdir(rj_path) if file.startswith('REPORTE_CORTES_BBVA_') and file.endswith('.xlsx')]
clasa_archivos = [file for file in os.listdir(clasa_path) if file.startswith('REPORTE_CORTES_BBVA_') and file.endswith('.txt')]
mornese_archivos = [file for file in os.listdir(mornese_path) if file.startswith('REPORTE_CORTES_BBVA_') and file.endswith('.xlsx')]

print(rj_archivos)
print(clasa_archivos)
print(mornese_archivos)

In [ ]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('.', '').replace(' ', '_').upper() for column in columns_list]

In [ ]:
def detectar_codificacion(ruta_archivo):
    with open(ruta_archivo, 'rb') as f:
        resultado = chardet.detect(f.read())
    return resultado['encoding']

In [ ]:
def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    if 'AGENCIA' in df.columns:
        df['AGENCIA'] = df['AGENCIA'].str.upper().replace('ASESCOM RJ ABOGADOS', 'ASESCOM RJ')
    if 'TIPO_DE_MARCACIÓN' in df.columns:
        df.rename(columns={'TIPO_DE_MARCACIÓN': 'TIPO_MARCADOR'}, inplace=True)
    if 'TIPO_DE_DISCADO' in df.columns:
        df.rename(columns={'TIPO_DE_DISCADO': 'TIPO_MARCADOR'}, inplace=True)
    if 'PRIORIDAD_TELEFONO' not in df.columns:
        df['PRIORIDAD_TELEFONO'] = np.nan
    
    df['MARCA'] = df['MARCA'].str.strip().str.upper().replace(' ', '', regex=True)
    df['MARCA'] = np.where(df['MARCA'] == 'KST', 'KSTBC', df['MARCA'])
    df = df[df['MARCA'].isin(['BC', 'EXJ', 'KSTBC'])]
    df['FECHA_LLAMADA'] = pd.to_datetime(df['FECHA_LLAMADA'], errors='coerce')
    df['HORA_INICIO_LLAMADA'] = pd.to_datetime(df['HORA_INICIO_LLAMADA']).dt.hour.astype('Int64').fillna(0)
    df['TIPO_CONTACTO'] = df['TIPO_CONTACTO'].str.upper().replace('TERCERO', 'CONTACTO INDIRECTO').replace('CONTACTO EFECTIVO', 'CONTACTO DIRECTO').replace('EFECTIVO', 'CONTACTO DIRECTO')
    df['TRAMO'] = df['TRAMO'].str.strip().str.upper().replace(' ', '', regex=True).replace(['T1', 'T2', 'T3', 'T4'], 'NULL').fillna('NULL')
    df['PDP'] = df.apply(lambda x: 1 if pd.notna(x['FECHA_PDP']) else 0, axis=1)
    df['TIPO_MARCADOR'] = df['TIPO_MARCADOR'].str.strip().str.upper().replace(' ', '', regex=True).fillna('NULL')
    
    df = df[['AGENCIA', 'COD_CENTRAL', 'CONTRATO', 'MARCA', 'FECHA_LLAMADA', 'HORA_INICIO_LLAMADA', 'TIPO_CONTACTO', 'TRAMO', 'PDP', 'TIPO_MARCADOR', 'PRIORIDAD_TELEFONO']]
    return df

In [ ]:
def consolidar_archivos(agencia_path: str, archivos: list) -> pd.DataFrame:
    dataframes = []
    for archivo in archivos:
        ruta_completa = os.path.join(agencia_path, archivo)
        if archivo.endswith('.txt'):
            df = pd.read_csv(
                ruta_completa, 
                sep=',', 
                dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str}, 
                encoding='ISO-8859-1'
                )
        else:
            df = pd.read_excel(
                ruta_completa, 
                dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str}
                )
        
        df.columns = clean_columns(df.columns)
        dataframes.append(df)
        print(f'{archivo} leído correctamente')
    
    df_consolidado = pd.concat(dataframes, ignore_index=True)
    df_consolidado = clean_dataframe(df_consolidado)
    
    df_consolidado.to_csv(
        os.path.join(agencia_path, f'CONSOLIDADO_GESTIONES_{agencia_path.split("/")[-1]}.csv'), 
        index=False, 
        sep='|'
        )
    
    return df_consolidado

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Consolidar archivos de gestiones?', icon='warning')

if result == 'yes':
    dataframes = {}
    archivos = {
        'RJ': (rj_path, rj_archivos),
        'CLASA': (clasa_path, clasa_archivos),
        #'MORNESE': (mornese_path, mornese_archivos)
    }
    
    for key, (path, file_list) in archivos.items():
        if file_list:
            dataframes[key] = consolidar_archivos(path, file_list)
            print(f'Consolidado {key}:', dataframes[key].shape)
        else:
            print(f'No hay archivos para consolidar en {key}.')

root.destroy()

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Leer consolidados?', icon='warning')

if result == 'yes':
    dataframes = {}
    archivos = {
        'RJ': f'{rj_path}/CONSOLIDADO_GESTIONES_RJ.csv',
        'CLASA': f'{clasa_path}/CONSOLIDADO_GESTIONES_CLASA.csv',
        #'MORNESE': f'{mornese_path}/CONSOLIDADO_GESTIONES_MORNESE.csv'
    }
    
    for key, file_path in archivos.items():
        if os.path.exists(file_path):
            dataframes[key] = pd.read_csv(
                file_path, 
                sep='|', 
                dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str}, 
                encoding='ISO-8859-1'
                )
            print(f'Consolidado {key} leído:', dataframes[key].shape)
        else:
            print(f'El archivo CONSOLIDADO_GESTIONES_{key}.csv no existe.')

root.destroy()

In [ ]:
df_rj_test: pd.DataFrame = dataframes['RJ'].copy()
df_clasa_test: pd.DataFrame = dataframes['CLASA'].copy()
#df_mornese_test: pd.DataFrame = dataframes['MORNESE'].copy()

print(df_rj_test.columns)
print(df_clasa_test.columns)

In [ ]:
cod_central_counts = df_rj_test.groupby('COD_CENTRAL')['PERIODO'].nunique()
cod_central_in_three_months = cod_central_counts[cod_central_counts == 3].index
cod_central_in_three_months

In [ ]:
df_rj_test[df_rj_test['COD_CENTRAL'] == '31625450']

In [ ]:
df_rj_test['PERIODO'] = pd.to_datetime(df_rj_test['FECHA_LLAMADA']).dt.strftime('%Y%m').astype(str)
df_rj_test['INTENSITDAD_TOTAL'] = df_rj_test.groupby(['COD_CENTRAL', 'PERIODO'])['COD_CENTRAL'].transform('count')

In [ ]:
df_rj_test['DIRECTO_CALL'] = df_rj_test.groupby(['CONTRATO', 'PERIODO'])['CONTRATO'].transform('count')

df_rj_test['INTENSIDAD_DIRECTA'] = df_rj_test['DIRECTO_CALL'] / df_rj_test['DIRECTO_CALL']

In [ ]:
def add_new_columns(df: pd.DataFrame) -> pd.DataFrame:
    
    df['INTENSIDAD_TOTAL'] = np.nan
    df['INTENSIDAD_DIRECTA'] = np.nan
    df['COBERTURA'] = np.nan
    df['CONTACTO_DIRECTO'] = np.nan
    df['TASA_CIERRE'] = np.nan
    
    return df

In [ ]:
df_rj_indicadores = df_rj_test.groupby(['AGENCIA', 'FECHA_LLAMADA', 'MARCA', 'TRAMO', 'TIPO_MARCADOR', 'PRIORIDAD_TELEFONO']).agg({
    'INTENSIDAD_TOTAL': 'mean', 
    'INTENSIDAD_DIRECTA': lambda x: x[x != 0].mean(), 
    'COBERTURA': 'mean', 
    'CONTACTO_DIRECTO': lambda x: x.mean(skipna=True), 
    'TASA_CIERRE': lambda x: x.mean(skipna=True), 
    'CALIDAD_PROMESAS': lambda x: x.mean(skipna=True)
}).reset_index()

df_rj_indicadores.head()

In [ ]:
df_rj_indicadores = df_rj_test.groupby(['AGENCIA', 'MARCA', 'FECHA_LLAMADA', 'TIPO_CONTACTO', 'TRAMO', 'PDP', 'TIPO_MARCADOR', 'PRIORIDAD_TELEFONO']).agg({
    'INTENSIDAD_TOTAL': 'mean', 
    'INTENSIDAD_DIRECTA': lambda x: x[x != 0].mean(), 
    'COBERTURA': 'mean', 
    'CONTACTABILIDAD_DIRECTA': lambda x: x.mean(skipna=True), 
    'TASA_CIERRE': lambda x: x.mean(skipna=True), 
    'CALIDAD_PROMESAS': lambda x: x.mean(skipna=True)
}).reset_index()

df_rj_indicadores['INTENSIDAD_TOTAL'] = df_rj_indicadores['INTENSIDAD_TOTAL'].round(2)
df_rj_indicadores['INTENSIDAD_DIRECTA'] = df_rj_indicadores['INTENSIDAD_DIRECTA'].round(2)
df_rj_indicadores['COBERTURA'] = df_rj_indicadores['COBERTURA'].round(4)
df_rj_indicadores['CONTACTABILIDAD_DIRECTA'] = df_rj_indicadores['CONTACTABILIDAD_DIRECTA'].round(2)
df_rj_indicadores['TASA_CIERRE'] = df_rj_indicadores['TASA_CIERRE'].round(2)
df_rj_indicadores['CALIDAD_PROMESAS'] = df_rj_indicadores['CALIDAD_PROMESAS'].round(2)

df_rj_indicadores.sort_values(by=['PERIODO', 'CARTERA', 'AGENCIA'], inplace=True)
df_rj_indicadores

In [ ]:
print(df_clasa_test['MARCA'].value_counts(dropna=False))
print('---')
print(df_rj_test['MARCA'].value_counts(dropna=False))

In [ ]:
print(df_clasa_test['TIPO_CONTACTO'].value_counts(dropna=False))
print('---')
print(df_rj_test['TIPO_CONTACTO'].value_counts(dropna=False))

In [ ]:
print(df_clasa_test['TRAMO'].value_counts(dropna=False))
print('---')
print(df_rj_test['TRAMO'].value_counts(dropna=False))

In [ ]:
print(df_clasa_test['PDP'].value_counts(dropna=False))
print('---')
print(df_rj_test['PDP'].value_counts(dropna=False))

In [ ]:
print(df_clasa_test['TIPO_MARCADOR'].value_counts(dropna=False))
print('---')
print(df_rj_test['TIPO_MARCADOR'].value_counts(dropna=False))